In [5]:
# Assume farm_data is a dictionary outputted by data_loader. 
# farm_data = data_loader(farm_id)
# Here is just a toy example.
test_data = {
        "group": "annual",
        "yield": 3500,
        "carbon_concentration": 0.45,
        "moisture": 14,
        "area": 10,
        "S_p": 2,
        "S_s": 100,
        "S_r": 100,
        "R_p": 0.304,
        "R_s": 0.455,
        "R_r": 0.146,
        "R_e": 0.095,
        "N_p": 67,
        "N_s": 6,
        "N_r": 10,
        "N_e": 10
    }

In [7]:
class CropResidueCalculator:
    def __init__(self, farm_data):

# should I use self.farm_data = farm_data then use self.farm_data["area"] 
# or assign each value to a self.variable like self.area = farm_data["area"]?
        self.farm_data = farm_data
        self.area = farm_data["area"]
        self.group = farm_data["group"]
        self.crop_yield = farm_data["yield"]
        self.moisture = farm_data["moisture"]
        self.carbon_concentration = farm_data["carbon_concentration"]
        

        self.S_p = farm_data["S_p"]
        self.S_s = farm_data["S_s"]
        self.S_r = farm_data["S_r"]

        self.R_p = farm_data["R_p"]
        self.R_s = farm_data["R_s"]
        self.R_r = farm_data["R_r"]
        self.R_e = farm_data["R_e"]

# should devide by 1000 to convert to kg here or in the functions?
        self.N_p = farm_data["N_p"]
        self.N_s = farm_data["N_s"]
        self.N_r = farm_data["N_r"]
        self.N_e = farm_data["N_e"]



        
    def c_p(self):
        return (self.crop_yield + self.crop_yield * self.S_p / 100) * (1 - self.moisture / 100) * self.carbon_concentration
    
    def grain_n(self):
        C_p_to_soil = self.c_p() * self.S_p / 100
        return C_p_to_soil / 0.45 * (self.N_p / 1000)
    
    def straw_n(self):
        C_s = self.c_p() * (self.R_s / self.R_p) * (self.S_s / 100)
        return C_s / 0.45 * (self.N_s / 1000)

    def root_n(self):
        C_r = self.c_p() * (self.R_r / self.R_p) * (self.S_r / 100)
        return C_r / 0.45 * (self.N_r / 1000)

    def exudate_n(self):
        C_e = self.c_p() * (self.R_e / self.R_p) 
        return C_e / 0.45 * (self.N_e /1000)

    def above_ground_residue_n(self):
        if self.group in ["annual", "perennial"]:
            return self.grain_n() + self.straw_n()
        elif self.group == "root":
            return self.straw_n()
        elif self.group in ["cover", "silage"]:
            return self.grain_n()
        else:
            return 0

        
    def below_ground_residue_n(self):
        if self.group == "annual":
            return self.root_n() + self.exudate_n()
        elif self.group == "perennial":
            return self.root_n() * (S_r / 100) + self.exudate_n()
        elif self.group == "root":
            return self.grain_n() + self.exudate_n()
        elif self.group in ["cover", "silage"]:
            return self.root_n() + self.exudate_n()
        else:
            return 0

    def n_crop_residue(self):
        return (self.above_ground_residue_n() + self.below_ground_residue_n()) * self.area

In [10]:
calculator = CropResidueCalculator(test_data)
print("Total Above Ground Residue Nitrogen:", calculator.above_ground_residue_n())
print("Total Below Ground Residue Nitrogen:", calculator.below_ground_residue_n())
print("Total Nitrogen Crop Residue:", calculator.n_crop_residue())

Total Above Ground Residue Nitrogen: 31.685271947368424
Total Below Ground Residue Nitrogen: 24.339414473684208
Total Nitrogen Crop Residue: 560.2468642105264
